# Combining Datasets

Before doing any feature generation or modelling we need to combine all the datasets we've fetched into one dataset organised by household.

In [ ]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
# Need to move up to parent directory to import local functions
os.chdir("..")

from hmo_identifier.process import merge, address

## Gazatteer

The gazatteer is our base data that we will use the merge other datasets onto. This is the gazatteer for Camden

In [ ]:
gaz = pd.read_csv("data/raw/local/gazatteer.csv")
gaz.head()

We can join a lot of the datasets spatially, so we need to convert the pandas dataframe to a geopandas (spatial) dataframe so we can spatially join to other
datasets.
Careful of the coordinate reference system (CRS) - this gazatteer is in EPSG:27700 (British National Grid, Eastings/Northings) but a lot of the other datasets will be in EPSG:4326 (latitude/longitude)

In [ ]:
gaz = gpd.GeoDataFrame(gaz,
                       geometry=gpd.points_from_xy(gaz.x_coordinate,
                                                   gaz.y_coordinate),
                       crs=27700)

In [ ]:
gaz.head()

Each row of the gazatteer now has a spatial location associated with it.


In [ ]:
f, ax = plt.subplots(1)
ax = gaz.plot(ax=ax, markersize=0.25)
ax.set_axis_off()
plt.show()

Now that we've got a spatial dataframe we can add some reference geographies. These files includes census ouput areas and wards

In [ ]:
output_areas = gpd.read_file("data/raw/reference/output_areas.shp")
output_areas = output_areas.to_crs(27700)
wards = gpd.read_file("data/raw/reference/wards.shp")
wards = wards.to_crs(27700)
for ref_geo in [output_areas, wards] :
     gaz = (gpd.sjoin(gaz, ref_geo, how='left')
            .drop(columns='index_right'))

We now have the ward and census output area for each gazatteer entry.

## Joining by UPRN

The most reliable way to join in additional datasets is by UPRN. Any datasets that could include a UPRN should be linked to the gazatteer in this way. A helper function to do this is included in hmo_identifier.process.merge

The UK Buildings data, which contains information on building structure, includes UPRN so we can start by adding this to the gazatteer.

The UK Buildings data comes with a "link file", which contains the relationships between the identifier used in this dataset (UPN/unique building number) and UPRN. We need to merge this into the UK Buildings data before we can join in to the gazatteer

In [ ]:
ukb = pd.read_csv("data/raw/local/uk_buildings.csv")
ukb_uprn = pd.read_csv('data/raw/local/uk_buildings_link.csv')
ukb = (ukb
       .merge(ukb_uprn)
       .drop(["ubn", "upn"], axis = 1)
      .drop_duplicates())

gaz = merge.by_uprn(gaz, ukb, "ukb")

The gazatteer now has additional columns from UK buildings.

In [ ]:
gaz.head()

Even with UPRN, the merge won't be perfect:

In [ ]:
gaz.merge_ukb.value_counts()

## Joining by Geographic boundaries

Data that is aggregated to different geographic levels can be merged onto the gazatteer.

Census data is at the output area level, and IMD is at LSOA. We added geographic levels to the gazatteer earlier, and we can use the helper function merge.by_geog to add the census and IMD data, we just need to get the geography columns into the right format

In [ ]:
census = pd.read_csv("data/raw/open/census.csv")
census = (census
          .drop(columns=['geography', 'date'])
          .rename(columns={'geography_code': 'oacd'}))
gaz = merge.by_geog(gaz, census, name="census")
imd = pd.read_csv('data/raw/open/imd.csv')
imd = imd.drop(columns=['ladcd', 'ladnm', 'lsoanm'])
gaz = merge.by_geog(gaz, imd, name='imd')

In [ ]:
gaz.head()

We should have a full match for these matches as they are location based.
TODO: Sort out boundary cases

In [ ]:
gaz.merge_imd.value_counts()

In [ ]:
f, ax = plt.subplots(1)
ax = gaz.plot(ax=ax, markersize=0.25, column="imd_decile_imd")
ax.set_axis_off()
plt.show()

## Joining spatial point data

Some datasets contain data on a specific location, but not necessarily a specific household. For example, the crime data contains a point location for the crime reported, but this could be on a street or in a park. To join this into our gazatteer we can summarise the data within the area around the household.



In [ ]:
crime = pd.read_csv("data/raw/open/crime.csv")
crime = gpd.GeoDataFrame(crime, geometry = gpd.points_from_xy(crime.location_longitude, crime.location_latitude),
                         crs = 4326)
crime = crime.to_crs(27700)
crime.head()

In [ ]:
crime.category.value_counts()

We need to create indicators for different types of crime

In [ ]:
crime = (crime
         .assign(asb = crime.category.isin(['anti-social-behaviour', 'public-order']),
                 violent = crime.category.isin(['violent-crime', 'possession-of-weapons']),
                 theft = crime.category.isin(['other-theft', 'theft-from-the-person',
                                              'burglary', 'robbery',
                                              'shoplifting', 'bicycle-theft']),
                 other = crime.category.isin(['vehicle-crime', 'drugs',
                                              'criminal-damage-arson', 'other-crime'])))
crime = crime[["asb", "violent", "theft", "other", "geometry"]]

merge.by_buffer will summarise the new data in an area around each household (size specified by buffer)

In [ ]:
gaz = merge.by_buffer(ref = gaz, add = crime, name = "crime", buffer = 200,
                           sum_cols = ["sum"])

We now have an indicator of the previlence of different crime types around households

In [ ]:
f, ax = plt.subplots(1)
ax = gaz.plot(ax=ax, markersize=0.25, column='asb_sum_crime')
ax.set_axis_off()
plt.show()

And we can apply the same method to the airbnb data

In [ ]:
airbnb = pd.read_csv('data/raw/open/airbnb.csv')
airbnb = gpd.GeoDataFrame(airbnb,
                          geometry=gpd.points_from_xy(airbnb.longitude,
                                                      airbnb.latitude),
                          crs=4326)
airbnb = (airbnb
          .to_crs(27700)
          .assign(price_pp=(airbnb.price
                            .str.replace("^\$|\.00$|,", "")
                            .astype(int)
                            / airbnb.accommodates))
          [['price_pp', 'accommodates', 'geometry']])
gaz = merge.by_buffer(ref = gaz, add = airbnb, name = "abnb", buffer = 200,
                           sum_cols = ['median'])

In [ ]:
f, ax = plt.subplots(1)
ax = gaz.plot(ax=ax, markersize=0.25, column='price_pp_median_abnb')
ax.set_axis_off()
plt.show()

In [ ]:
f, ax = plt.subplots(1)
ax = gaz.plot(ax=ax, markersize=0.25, column='accommodates_median_abnb')
ax.set_axis_off()
plt.show()

## Address Matching

Some datasets are at the household level, but don't contain a UPRN. To join these datasets to the gazatteer we need to address match them.

As part of the initial data processing, a version of the gazatteer was produced with the 2 different forms of address it incudes, geographical address and delivery point address

In [ ]:
gaz_add = pd.read_csv("data/interim/gazatteer_address.csv")
gaz_add.head()

In [ ]:
gaz_add = gaz_add.loc[gaz_add.uprn.isin(gaz.uprn),:]
gaz = pd.merge(gaz, gaz_add[['uprn', 'geo_address','postcode']], how='left')

In [ ]:
epc = pd.read_csv("data/raw/open/epc.csv", usecols = ['brn', 'address', 'postcode', 'lodgement_date'])
epc.head()

The EPC data contains a row for each time a property was given an EPC, so there could be multiple for each property. We only need to address match once per property, so lets use the most recent.

In [ ]:
epc = (epc
       .sort_values(by='lodgement_date', ascending=False)
       .drop(columns='lodgement_date')
       .drop_duplicates('brn'))

In [ ]:
epc = address.match_prep(epc, add_var = 'address')
epc.head()

In [ ]:
gaz_geo = (address.match_prep(gaz_add, add_var='geo_address')
          .drop(columns='dp_address')
          .rename(columns={'clean_address': 'clean_address_geo',
                          'numbers': 'numbers_geo',
                          'clean_address_flat': 'clean_address_flat_geo'}))
gaz_dp = (address.match_prep(gaz_add.loc[~pd.isna(gaz_add.dp_address), :],
                             add_var='dp_address')
         .drop(columns='geo_address')
         .rename(columns={'clean_address': 'clean_address_dp',
                          'numbers': 'numbers_dp',
                          'clean_address_flat': 'clean_address_flat_dp'}))
gaz_add = pd.merge(gaz_geo, gaz_dp, how="outer", on=['uprn', 'postcode']).fillna("")
gaz_add.head()

In [ ]:
possible_matches = address.candidate_matches(ref=gaz_add, ref_id='uprn',
                                             ref_addresses=['clean_address_geo', 'clean_address_dp'],
                                            add=epc, add_id='brn', add_addresses=['clean_address'])
possible_matches.head()

In [ ]:
possible_matches.sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False).head()

In [ ]:
matches = (possible_matches.loc[(possible_matches.numbers_geo_numbers_match == 1) &
                                (possible_matches.clean_address_geo_clean_address_match > 0.7),:]
           .sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False)
          .drop_duplicates('uprn')
          .drop_duplicates('brn'))
matches.tail(10)

In [ ]:
matches = (possible_matches.loc[(possible_matches.numbers_geo_numbers_match == 1) &
                                (possible_matches.clean_address_geo_clean_address_match > 0.703),:]
           .sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False)
          .drop_duplicates('uprn')
          .drop_duplicates('brn'))
matches.tail(10)

In [ ]:
matched = matches.copy()
possible_matches = possible_matches.loc[~possible_matches.uprn.isin(matched.uprn),:]
possible_matches = possible_matches.loc[~possible_matches.brn.isin(matched.brn),:]


In [ ]:
possible_matches.sort_values(by=['numbers_dp_numbers_match', 'clean_address_dp_clean_address_match'], ascending=False).head(20)

In [ ]:
matches = (possible_matches.loc[(possible_matches.numbers_geo_numbers_match == 1) &
                                (possible_matches.clean_address_geo_clean_address_match > 0.58) &
                                (possible_matches.clean_address_dp_clean_address_match > 0.7),:]
           .sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False)
          .drop_duplicates('uprn')
          .drop_duplicates('brn'))
matches.tail(10)

In [ ]:
matched = pd.concat([matched, matches])
possible_matches = possible_matches.loc[~possible_matches.uprn.isin(matched.uprn),:]
possible_matches = possible_matches.loc[~possible_matches.brn.isin(matched.brn),:]


In [ ]:
possible_matches.sort_values(by=['numbers_dp_numbers_match', 'clean_address_dp_clean_address_match'], ascending=False).head(20)

In [ ]:
uprn_brn_lookup = matched[['uprn', 'brn']]

In [ ]:
epc = pd.read_csv("data/raw/open/epc.csv")

In [ ]:
gaz = pd.merge(gaz, uprn_brn_lookup, how="left")

In [ ]:
latest_epc = (epc.copy()
       .sort_values(by='lodgement_date', ascending=False)
       .drop_duplicates('brn')
       .drop(columns=['lmk_key', 'address1', 'address2', 'address3', 'postcode',
                   'inspection_date', 'local_authority', 'constituency',
                    'county', 'mechanical_ventilation', 'address',
                    'local_authority_label', 'constituency_label']))
latest_epc.columns = latest_epc.columns + "_epc"
latest_epc = latest_epc.rename(columns={'brn_epc': 'brn'})

In [ ]:
gaz = pd.merge(gaz, latest_epc, how='left', on='brn')

In [ ]:
gaz.head()

In [ ]:
f, ax = plt.subplots(1)
ax = gaz.plot(ax=ax, markersize=0.25, column='total_floor_area_epc')
ax.set_axis_off()
plt.show()

In [ ]:
epc_summary = (epc.copy()
 .groupby('brn').lmk_key
 .count()
.reset_index().
rename(columns={'lmk_key': 'no_entries_epc'}))
epc_summary.head()

In [ ]:
gaz = pd.merge(gaz, epc_summary, how='left', on='brn')

And we can go through the same process with the Land Registry data

In [ ]:
lr = pd.read_csv("data/raw/open/land_registry.csv",
                 usecols=['trans_id','paon', 'saon', 'street', 'postcode', 'date'])

In [ ]:
lr.head()

In [ ]:
lr = lr.fillna("")
lr['address'] = lr[['saon', 'paon', 'street']].apply(lambda x: " ".join(x), axis = 1)
lr = address.match_prep(lr, add_var='address')

In [ ]:
lr.head()

In [ ]:
lr = (lr
      .sort_values('date')
      .drop_duplicates('clean_address')
     .drop(columns='date'))

In [ ]:
possible_matches = address.candidate_matches(ref=gaz_add, ref_id='uprn',
                                             ref_addresses=['clean_address_geo', 'clean_address_dp'],
                                            add=lr, add_id='trans_id', add_addresses=['clean_address'])
possible_matches.head()

In [ ]:
possible_matches.sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False).head()

In [ ]:
matches = (possible_matches.loc[(possible_matches.numbers_geo_numbers_match == 1) &
                                (possible_matches.clean_address_geo_clean_address_match > 0.703),:]
           .sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False)
          .drop_duplicates('uprn')
          .drop_duplicates('trans_id'))
matches.tail(10)

In [ ]:
matched = matches.copy()
possible_matches = possible_matches.loc[~possible_matches.uprn.isin(matched.uprn),:]
possible_matches = possible_matches.loc[~possible_matches.trans_id.isin(matched.trans_id),:]


In [ ]:
matches = (possible_matches
           .sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False)
          .drop_duplicates('uprn')
          .drop_duplicates('trans_id'))
matches.tail(10)

In [ ]:
uprn_transid_lookup = matched[['uprn', 'trans_id']]

In [ ]:
lr = pd.read_csv("data/raw/open/land_registry.csv",
                 usecols = ['trans_id', 'price', 'date', 'prop_type', 'new_build', 'tenure_duration', 'ppd_cat', 'status'])
lr.columns = lr.columns + "_lr"
lr = lr.rename(columns={'trans_id_lr': 'trans_id'})
gaz = pd.merge(gaz, uprn_transid_lookup, how = 'left')
gaz = pd.merge(gaz, lr, how='left', on=['trans_id'])
gaz.head()

In [ ]:
f, ax = plt.subplots(1)
ax = gaz.plot(ax=ax, markersize=0.25, column='price_lr')
ax.set_axis_off()
plt.show()

### More approximate address matching

Some datasets may not contain the exact address. We therefore need to do a rougher adddress match to add in these datasets.

In [ ]:
social_housing = pd.read_csv("data/raw/local/social_housing.csv")
social_housing.head()

In [ ]:
social_housing = social_housing[['estate_name', 'ward_name']].drop_duplicates().dropna()
#social_housing = social_housing.loc[social_housing.ward_name.str.contains("gospel oak|haverstock", case=False), :]
social_housing = social_housing.loc[social_housing.estate_name != "-", :]
social_housing.head(20)

In [ ]:
gaz_add_ward = pd.merge(gaz_add[['uprn', 'clean_address_geo', 'clean_address_dp']], gaz[['uprn', 'wardnm']])
gaz_add_ward.head(20)

In [ ]:
social_housing = social_housing.assign(wardnm = social_housing.ward_name.str.replace(" Ward", ""),
                                      clean_estate_name = address.clean_estate_name(social_housing.estate_name))
social_housing.head(20)

In [ ]:
social_housing.clean_estate_name = social_housing.clean_estate_name.str.lower()
gaz_add_ward['geo_address_list'] = gaz_add_ward.clean_address_geo.str.split()
gaz_add_ward['dp_address_list'] = gaz_add_ward.clean_address_dp.str.split()
social_housing['estate_list'] = social_housing.clean_estate_name.str.split()
gaz_estates = pd.merge(gaz_add_ward, social_housing, how = 'inner')
gaz_estates.head()

In [ ]:
gaz_estates["estate_in_geo"] = gaz_estates.apply(lambda x: all(e in x.geo_address_list for e in x.estate_list), axis=1)
gaz_estates["estate_in_dp"] = gaz_estates.apply(lambda x: all(e in x.dp_address_list for e in x.estate_list), axis=1)
gaz_estates.head(20)


In [ ]:
gaz_estates = (gaz_estates.loc[(gaz_estates.estate_in_geo) | (gaz_estates.estate_in_dp), "uprn"]
               .drop_duplicates()
               .tolist())

gaz['social_housing'] = gaz.uprn.isin(gaz_estates)

## HMO Register

In [ ]:
hmos = pd.read_csv("data/raw/local/hmo_register.csv", usecols=['licence_number', 'property_address'])

In [ ]:
hmos['address'] = hmos.property_address.str.replace("Greater London", "").str.replace("\\bLondon\\b ", "").str.replace("[A-Z]{1,2}[0-9][A-Z0-9]? ?[0-9][A-Z]{2}", "")
hmos['postcode'] = hmos.property_address.str.extract("([A-Z]{1,2}[0-9][A-Z0-9]? ?[0-9][A-Z]{2})", expand=False)
hmos = hmos.drop(columns='property_address').drop_duplicates().reset_index(drop=True)
hmos.shape

In [ ]:
hmos.head()

In [ ]:
hmos = address.match_prep(hmos, add_var = 'address')
hmos.head()

In [ ]:
possible_matches = address.candidate_matches(ref=gaz_add, ref_id='uprn',
                                             ref_addresses=['clean_address_geo', 'clean_address_dp'],
                                            add=hmos, add_id='licence_number', add_addresses=['clean_address'])
possible_matches.head()

In [ ]:
possible_matches.sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False).head()

We're not going to drop licence_number duplicates as several flats can be covered by one HMO licence

In [ ]:
matches = (possible_matches.loc[(possible_matches.numbers_geo_numbers_match == 1),:]
           .sort_values(by=['numbers_geo_numbers_match', 'clean_address_geo_clean_address_match'], ascending=False)
          .drop_duplicates('uprn'))
matches.tail(10)

In [ ]:
matched = matches.copy()
possible_matches = possible_matches.loc[~possible_matches.uprn.isin(matched.uprn),:]
possible_matches = possible_matches.loc[~possible_matches.licence_number.isin(matched.licence_number),:]

In [ ]:
matches = possible_matches[possible_matches.apply(lambda row: row.clean_address in row.clean_address_geo, axis=1)]

In [ ]:
matches

In [ ]:
matched = pd.concat([matched, matches])
possible_matches = possible_matches.loc[~possible_matches.uprn.isin(matched.uprn),:]
possible_matches = possible_matches.loc[~possible_matches.licence_number.isin(matched.licence_number),:]

In [ ]:
gaz['hmo'] = gaz.uprn.isin(matched.uprn)

In [ ]:
gaz.drop(columns='geometry').to_csv("data/interim/gazatteer_combined.csv", index=False)